# Importing Libraries

In [151]:
import requests
import csv
import os
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer 
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.multivariate.manova import MANOVA
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Scraping Reviews from Amazon: place, title, body, rating

In [3]:
#JBL
class AmazonScrap:
    def __init__(self,url):
        self.url="https://www.amazon.in/JBL-Playtime-Charging-Headphones-Assistant/product-reviews/B096G2RN6D/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1"
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0 (Edition std-1)'}
    
    def get_reviews(self,max_pages=20):
        reviews=[]
        page_nu=1
        while page_nu<=max_pages:
            response = requests.get(f'{self.url}&pageNumber={page_nu}', headers=self.headers)
            if response.status_code!=200:
                break
            
            soup=BeautifulSoup(response.content,'html.parser')
            
            review_blocks = soup.find_all('div', {'data-hook': 'review'})
            if not review_blocks:
                break
            
            for review_block in review_blocks:
                rating=review_block.find('i',{'data-hook': 'review-star-rating'}).text.strip().split(' ')[0]
                title = review_block.find('a', {'data-hook': 'review-title'}).text.strip()
                body = review_block.find('span', {'data-hook': 'review-body'}).text.strip()
                date_place=review_block.find('span',{'data-hook':"review-date"}) .text.strip()
                reviews.append({
                    'rating': rating,
                    'title': title,
                    'body': body,
                    'place_of_review':date_place
                })
            page_nu+=1
          
        return reviews
    
    def save_reviews(self, reviews, filename):
        # Save the reviews to a CSV file
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Rating', 'Title', 'Body','place_of_review'])
            for review in reviews:
                writer.writerow([review['rating'], review['title'], review['body'],review['place_of_review']])

if __name__ == '__main__':
    # Enter the URL of the Amazon product reviews page
    url = 'https://www.amazon.in/JBL-Playtime-Charging-Headphones-Assistant/product-reviews/B096G2RN6D/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'
    
    # Create an instance of the AmazonReviewsScraper class and get the reviews
    scraper = AmazonScrap(url)
    reviews = scraper.get_reviews()
    
    # Save the reviews to a CSV file
    scraper.save_reviews(reviews, 'JBLreviews.csv')
    
    print(f'{len(reviews)} reviews scraped and saved to JBLreviews.csv')      

200 reviews scraped and saved to JBLreviews.csv


In [4]:
#Sony
class AmazonScrap:
    def __init__(self,url):
        self.url="https://www.amazon.in/SONY-WH-CH510-Wireless-Headphone-Blue/product-reviews/B0817T8FB6/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1"
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0 (Edition std-1)'}
    
    def get_reviews(self,max_pages=20):
        reviews=[]
        page_nu=1
        while page_nu<=max_pages:
            response = requests.get(f'{self.url}&pageNumber={page_nu}', headers=self.headers)
            if response.status_code!=200:
                break
            
            soup=BeautifulSoup(response.content,'html.parser')
            
            review_blocks = soup.find_all('div', {'data-hook': 'review'})
            if not review_blocks:
                break
            
            for review_block in review_blocks:
                rating=review_block.find('i',{'data-hook': 'review-star-rating'}).text.strip().split(' ')[0]
                title = review_block.find('a', {'data-hook': 'review-title'}).text.strip()
                body = review_block.find('span', {'data-hook': 'review-body'}).text.strip()
                date_place=review_block.find('span',{'data-hook':"review-date"}) .text.strip()
                reviews.append({
                    'rating': rating,
                    'title': title,
                    'body': body,
                    'place_of_review':date_place
                })
            page_nu+=1
          
        return reviews
    
    def save_reviews(self, reviews, filename):
        # Save the reviews to a CSV file
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Rating', 'Title', 'Body','place_of_review'])
            for review in reviews:
                writer.writerow([review['rating'], review['title'], review['body'],review['place_of_review']])

if __name__ == '__main__':
    # Enter the URL of the Amazon product reviews page
    url = 'https://www.amazon.in/SONY-WH-CH510-Wireless-Headphone-Blue/product-reviews/B0817T8FB6/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'
    
    # Create an instance of the AmazonReviewsScraper class and get the reviews
    scraper = AmazonScrap(url)
    reviews = scraper.get_reviews()
    
    # Save the reviews to a CSV file
    scraper.save_reviews(reviews, 'Sonyreviews.csv')
    
    print(f'{len(reviews)} reviews scraped and saved to Sonyreviews.csv')      

200 reviews scraped and saved to Sonyreviews.csv


In [5]:
#Boat
class AmazonScrap:
    def __init__(self,url):
        self.url="https://www.amazon.in/boAt-Rockerz-558-Bluetooth-Headphones/product-reviews/B0BVRGZ9FV/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1"
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0 (Edition std-1)'}
    
    def get_reviews(self,max_pages=21):
        reviews=[]
        page_nu=1
        while page_nu<=max_pages:
            response = requests.get(f'{self.url}&pageNumber={page_nu}', headers=self.headers)
            if response.status_code!=200:
                break
            
            soup=BeautifulSoup(response.content,'html.parser')
            
            review_blocks = soup.find_all('div', {'data-hook': 'review'})
            if not review_blocks:
                break
            
            for review_block in review_blocks:
                rating=review_block.find('i',{'data-hook': 'review-star-rating'}).text.strip().split(' ')[0]
                title = review_block.find('a', {'data-hook': 'review-title'}).text.strip()
                body = review_block.find('span', {'data-hook': 'review-body'}).text.strip()
                date_place=review_block.find('span',{'data-hook':"review-date"}) .text.strip()
                reviews.append({
                    'rating': rating,
                    'title': title,
                    'body': body,
                    'place_of_review':date_place
                })
            page_nu+=1
          
        return reviews
    
    def save_reviews(self, reviews, filename):
        # Save the reviews to a CSV file
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Rating', 'Title', 'Body','place_of_review'])
            for review in reviews:
                writer.writerow([review['rating'], review['title'], review['body'],review['place_of_review']])

if __name__ == '__main__':
    # Enter the URL of the Amazon product reviews page
    url = 'https://www.amazon.in/boAt-Rockerz-558-Bluetooth-Headphones/product-reviews/B0BVRGZ9FV/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'
    
    # Create an instance of the AmazonReviewsScraper class and get the reviews
    scraper = AmazonScrap(url)
    reviews = scraper.get_reviews()
    
    # Save the reviews to a CSV file
    scraper.save_reviews(reviews, 'Boatreviews.csv')
    
    print(f'{len(reviews)} reviews scraped and saved to Boatreviews.csv')      

210 reviews scraped and saved to Boatreviews.csv


# Creating Dataframes 

In [122]:
df1=pd.read_csv('JBLreviews.csv', header=None, names=['rating','Title','Body','review_place'])
df1=df1.drop(0)
df1['rating']=df1['rating'].astype('float')
df1['country'] = df1['review_place'].str.split('Reviewed in ').str[1].str.split(' on ').str[0]
df1=df1.drop('review_place', axis=1)
df1['country'] = df1['country'].replace('India 🇮🇳', 'India')
df1=df1.dropna()
df1['Product']='JBL Tune 710BT'
df1

,rating,Title,Body,country,Product
1,4.0,Amazing noise cancellation,Not too loud over the ear audio experience. Cl...,India,JBL Tune 710BT
2,1.0,Poor quality,Very poor quality,India,JBL Tune 710BT
3,2.0,Sound quality.,"Very bad, the left side is having very low sou...",India,JBL Tune 710BT
4,4.0,Sound quality is great.,As expected the sound quality is amazing.Had s...,India,JBL Tune 710BT
5,1.0,Very poor🤢🤢,Sony is way way better,India,JBL Tune 710BT
...,...,...,...,...,...
196,5.0,Sound,"Clear sound quality nice , sound was amazing ...",India,JBL Tune 710BT
197,4.0,This is not for Bass lovers,The sound quality is excellent but it would ha...,India,JBL Tune 710BT
198,3.0,Not that great,This kinda On-Ear headphone only. It's not cov...,India,JBL Tune 710BT
199,4.0,Feels great!!,Sound and connectivity,India,JBL Tune 710BT


In [124]:
df1.isna().sum()

rating     0
Title      0
Body       0
country    0
Product    0
dtype: int64

In [125]:
df2=pd.read_csv('Sonyreviews.csv', header=None, names=['rating','Title','Body','review_place'])
df2=df2.drop(0)
df2['rating']=df2['rating'].astype('float')
df2['country'] = df2['review_place'].str.split('Reviewed in ').str[1].str.split(' on ').str[0]
df2=df2.drop('review_place', axis=1)
df2['country'] = df2['country'].replace('India 🇮🇳', 'India')
df2['Product']= "Sony Wh-Ch510"
df2

,rating,Title,Body,country,Product
1,3.0,Echo problem,Echo problem,India,Sony Wh-Ch510
2,1.0,To small,Item over the ear not perfect,India,Sony Wh-Ch510
3,5.0,Sony is Sony,Sony can’t be beaten by any other sounding pro...,India,Sony Wh-Ch510
4,5.0,Heavy duty and perfect sound,"I bought this product from reliance digital, 3...",India,Sony Wh-Ch510
5,1.0,Echo through mic when on call; worst customer ...,Echo happens through mic when on online/phone ...,India,Sony Wh-Ch510
...,...,...,...,...,...
196,5.0,Super product,It’s awesome product ..and the colour I like more,India,Sony Wh-Ch510
197,1.0,Feedback/Echo from mic calls for a Noise Cance...,All the surrounding Noise not only been heard ...,India,Sony Wh-Ch510
198,4.0,Worth it,Good sound quality and comfortable,India,Sony Wh-Ch510
199,5.0,Awesome Headset. Value for Money.,"If you are looking for a good quality headset,...",India,Sony Wh-Ch510


In [127]:
df2.isna().sum()

rating     0
Title      0
Body       0
country    0
Product    0
dtype: int64

In [128]:
df3=pd.read_csv('Boatreviews.csv', header=None, names=['rating','Title','Body','review_place'])
df3=df3.drop(0)
df3['rating']=df3['rating'].astype('float')
df3['country'] = df3['review_place'].str.split('Reviewed in ').str[1].str.split(' on ').str[0]
df3=df3.drop('review_place', axis=1)
df3['country'] = df3['country'].replace('India 🇮🇳', 'India')
df3=df3.dropna()
df3['Product']= "boAt Rockerz 558"
df3

,rating,Title,Body,country,Product
1,3.0,"Best Battery backup,good quality","But, heavy weight",India,boAt Rockerz 558
3,5.0,As always good,"Havr been buying boat headphones for years, al...",India,boAt Rockerz 558
4,5.0,"Super,worth of investing","Super,worth of investing",India,boAt Rockerz 558
5,4.0,sound,good for long term use,India,boAt Rockerz 558
6,5.0,"Wonderful product, great for long hours",Wonderful product. No issues wearing it over e...,India,boAt Rockerz 558
...,...,...,...,...,...
206,5.0,"Wonderful product, great for long hours",Wonderful product. No issues wearing it over e...,India,boAt Rockerz 558
207,1.0,Just RETURNED it,Well I bought this at ₹1999 from Amazon.The so...,India,boAt Rockerz 558
208,5.0,Great Buy,"Its preety good for its price point, has good...",India,boAt Rockerz 558
209,4.0,Uncomfortable but good sound quality and keeps...,i have been using the headphones for 5 days no...,India,boAt Rockerz 558


In [129]:
df3.isna().sum()

rating     0
Title      0
Body       0
country    0
Product    0
dtype: int64

In [131]:
#Dataframe with all 3 brands (not re-indexed)
dff=pd.concat([df1,df2,df3],axis=0)
dff

,rating,Title,Body,country,Product
1,4.0,Amazing noise cancellation,Not too loud over the ear audio experience. Cl...,India,JBL Tune 710BT
2,1.0,Poor quality,Very poor quality,India,JBL Tune 710BT
3,2.0,Sound quality.,"Very bad, the left side is having very low sou...",India,JBL Tune 710BT
4,4.0,Sound quality is great.,As expected the sound quality is amazing.Had s...,India,JBL Tune 710BT
5,1.0,Very poor🤢🤢,Sony is way way better,India,JBL Tune 710BT
...,...,...,...,...,...
206,5.0,"Wonderful product, great for long hours",Wonderful product. No issues wearing it over e...,India,boAt Rockerz 558
207,1.0,Just RETURNED it,Well I bought this at ₹1999 from Amazon.The so...,India,boAt Rockerz 558
208,5.0,Great Buy,"Its preety good for its price point, has good...",India,boAt Rockerz 558
209,4.0,Uncomfortable but good sound quality and keeps...,i have been using the headphones for 5 days no...,India,boAt Rockerz 558


# Sentiment Analysis to assign sentiment scores

In [133]:
nltk.download('punkt')

def preprocess_text(text):
    text = text.lower()  # convert to lowercase
    words = nltk.word_tokenize(text)  # tokenize the text into words
    words = [word for word in words if word.isalpha()]  # remove any non-alphabetic characters
    return ' '.join(words)  # join the list of words back into a string

dff['clean_title'] = dff['Title'].apply(preprocess_text)
dff['clean_body'] = dff['Body'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [134]:
nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    scores = analyzer.polarity_scores(text)
    return pd.Series(scores)

dff[['neg','neu','pos','comp']] = dff['clean_body'].apply(get_sentiment_scores)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [135]:
dff=dff.drop(['Title','Body'],axis=1)

In [136]:
dff

,rating,country,Product,clean_title,clean_body,neg,neu,pos,comp
1,4.0,India,JBL Tune 710BT,amazing noise cancellation,not too loud over the ear audio experience cle...,0.000,0.822,0.178,0.3818
2,1.0,India,JBL Tune 710BT,poor quality,very poor quality,0.629,0.371,0.000,-0.5256
3,2.0,India,JBL Tune 710BT,sound quality,very bad the left side is having very low soun...,0.279,0.721,0.000,-0.7340
4,4.0,India,JBL Tune 710BT,sound quality is great,as expected the sound quality is some noice wh...,0.000,1.000,0.000,0.0000
5,1.0,India,JBL Tune 710BT,very,sony is way way better,0.000,0.580,0.420,0.4404
...,...,...,...,...,...,...,...,...,...
206,5.0,India,boAt Rockerz 558,wonderful product great for long hours,wonderful product no issues wearing it over ea...,0.148,0.604,0.248,0.3612
207,1.0,India,boAt Rockerz 558,just returned it,well i bought this at from sound quality is ba...,0.098,0.718,0.184,0.4588
208,5.0,India,boAt Rockerz 558,great buy,its preety good for its price point has good s...,0.000,0.572,0.428,0.9136
209,4.0,India,boAt Rockerz 558,uncomfortable but good sound quality and keeps...,i have been using the headphones for days now ...,0.104,0.807,0.089,-0.6672


In [137]:
dff2=dff.drop(['clean_title','clean_body'],axis=1)

In [138]:
##Final dataframe with tone scores for each review
dff2

,rating,country,Product,neg,neu,pos,comp
1,4.0,India,JBL Tune 710BT,0.000,0.822,0.178,0.3818
2,1.0,India,JBL Tune 710BT,0.629,0.371,0.000,-0.5256
3,2.0,India,JBL Tune 710BT,0.279,0.721,0.000,-0.7340
4,4.0,India,JBL Tune 710BT,0.000,1.000,0.000,0.0000
5,1.0,India,JBL Tune 710BT,0.000,0.580,0.420,0.4404
...,...,...,...,...,...,...,...
206,5.0,India,boAt Rockerz 558,0.148,0.604,0.248,0.3612
207,1.0,India,boAt Rockerz 558,0.098,0.718,0.184,0.4588
208,5.0,India,boAt Rockerz 558,0.000,0.572,0.428,0.9136
209,4.0,India,boAt Rockerz 558,0.104,0.807,0.089,-0.6672


In [140]:
##Dataframe with 2 brands: Sony and JBL
dff3=dff2[(dff2['Product']=='Sony Wh-Ch510')|(dff2['Product']=='JBL Tune 710BT')]
dff3

,rating,country,Product,neg,neu,pos,comp
1,4.0,India,JBL Tune 710BT,0.000,0.822,0.178,0.3818
2,1.0,India,JBL Tune 710BT,0.629,0.371,0.000,-0.5256
3,2.0,India,JBL Tune 710BT,0.279,0.721,0.000,-0.7340
4,4.0,India,JBL Tune 710BT,0.000,1.000,0.000,0.0000
5,1.0,India,JBL Tune 710BT,0.000,0.580,0.420,0.4404
...,...,...,...,...,...,...,...
196,5.0,India,Sony Wh-Ch510,0.000,0.476,0.524,0.7650
197,1.0,India,Sony Wh-Ch510,0.048,0.827,0.126,0.8047
198,4.0,India,Sony Wh-Ch510,0.000,0.326,0.674,0.7351
199,5.0,India,Sony Wh-Ch510,0.000,0.791,0.209,0.7003


- categorical_var: Product, country, rating 
- continuous_var: pos, neg, neu, comp

# Manova and Anova Analysis

In [141]:
manova= MANOVA.from_formula('neg+neu+pos+comp~Product', data=dff2)
print(manova.mv_test())

                         Multivariate linear model
                                                                            
----------------------------------------------------------------------------
       Intercept            Value     Num DF  Den DF      F Value     Pr > F
----------------------------------------------------------------------------
          Wilks' lambda        0.0000 4.0000 583.0000 1853139648.1145 0.0000
         Pillai's trace        1.0000 4.0000 583.0000 1853139648.1145 0.0000
 Hotelling-Lawley trace 12714508.7349 4.0000 583.0000 1853139648.1145 0.0000
    Roy's greatest root 12714508.7349 4.0000 583.0000 1853139648.1145 0.0000
----------------------------------------------------------------------------
                                                                            
------------------------------------------------------------------------------
              Product           Value    Num DF     Den DF    F Value   Pr > F
---------------------

The MANOVA results indicate significant differences in sentiment scores (negative, neutral, positive, compound) between the products, as shown by Wilks' lambda, Pillai's trace, Hotelling-Lawley trace, and Roy's greatest root, all with p-values of 0.0000.

In [142]:
fit1= ols('neg~Product', data=dff2).fit()
anov1=sm.stats.anova_lm(fit1)
anov1

,df,sum_sq,mean_sq,F,PR(>F)
Product,2.0,1.360794,0.680397,21.662776,8.385313e-10
Residual,586.0,18.405429,0.031409,NaN,NaN


The ANOVA results show that the product categories significantly affects the negative sentiment scores, with a p-value of 8.385313e-10, indicating strong evidence against the null hypothesis of no difference between products.

In [143]:
fit2= ols('pos~Product', data=dff2).fit()
anov2=sm.stats.anova_lm(fit2)
anov2

,df,sum_sq,mean_sq,F,PR(>F)
Product,2.0,1.392868,0.696434,15.181221,3.732542e-07
Residual,586.0,26.882578,0.045875,NaN,NaN


The ANOVA results show that the product categories significantly affects the positive sentiment scores, with a p-value of 3.732542e-07, indicating strong evidence against the null hypothesis of no difference between products.

In [144]:
fit3= ols('neu~Product', data=dff2).fit()
anov3=sm.stats.anova_lm(fit3)
anov3

,df,sum_sq,mean_sq,F,PR(>F)
Product,2.0,2.104357,1.052179,23.900833,1.051144e-10
Residual,586.0,25.797289,0.044023,NaN,NaN


The ANOVA results show that the product categories significantly affects the neutral sentiment scores, with a p-value of 1.051144e-10, indicating strong evidence against the null hypothesis of no difference between products.

In [145]:
fit4= ols('comp~Product', data=dff2).fit()
anov4=sm.stats.anova_lm(fit4)
anov4

,df,sum_sq,mean_sq,F,PR(>F)
Product,2.0,8.642641,4.321321,15.310299,3.301560e-07
Residual,586.0,165.398068,0.282249,NaN,NaN


In [146]:
fit5= ols('comp~rating', data=dff).fit()
anov5=sm.stats.anova_lm(fit5)
anov5

,df,sum_sq,mean_sq,F,PR(>F)
rating,1.0,43.377628,43.377628,194.872704,1.891414e-38
Residual,587.0,130.663081,0.222595,NaN,NaN


In [147]:
x = MultiComparison(dff2['neg'], groups=dff2['Product'])
result=x.tukeyhsd()
print(result)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05          
    group1          group2      meandiff p-adj   lower   upper  reject
----------------------------------------------------------------------
JBL Tune 710BT    Sony Wh-Ch510   0.0511 0.0114  0.0095  0.0927   True
JBL Tune 710BT boAt Rockerz 558   -0.067 0.0006 -0.1093 -0.0248   True
 Sony Wh-Ch510 boAt Rockerz 558  -0.1181    0.0 -0.1604 -0.0759   True
----------------------------------------------------------------------



The Tukey HSD test shows significant differences in negative sentiment scores between all pairs of products, with Sony Wh-Ch510 having higher negative sentiment than JBL Tune 710BT and boAt Rockerz 558, and JBL Tune 710BT having higher negative sentiment than boAt Rockerz 558.

In [148]:
x2 = MultiComparison(dff2['pos'], groups=dff2['Product'])
result=x2.tukeyhsd()
print(result)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1          group2      meandiff p-adj   lower  upper  reject
---------------------------------------------------------------------
JBL Tune 710BT    Sony Wh-Ch510    0.091 0.0001  0.0407 0.1413   True
JBL Tune 710BT boAt Rockerz 558   0.1118    0.0  0.0608 0.1629   True
 Sony Wh-Ch510 boAt Rockerz 558   0.0208 0.6032 -0.0302 0.0719  False
---------------------------------------------------------------------


The Tukey HSD test indicates that the positive sentiment scores for the JBL Tune 710BT are significantly higher than both the Sony Wh-Ch510 and boAt Rockerz 558, while there is no significant difference between the Sony Wh-Ch510 and boAt Rockerz 558.

In [149]:
x3 = MultiComparison(dff2['neu'], groups=dff2['Product'])
result=x3.tukeyhsd()
print(result)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05          
    group1          group2      meandiff p-adj   lower   upper  reject
----------------------------------------------------------------------
JBL Tune 710BT    Sony Wh-Ch510   -0.142    0.0 -0.1913 -0.0927   True
JBL Tune 710BT boAt Rockerz 558  -0.0448 0.0895 -0.0948  0.0052  False
 Sony Wh-Ch510 boAt Rockerz 558   0.0972    0.0  0.0472  0.1472   True
----------------------------------------------------------------------


# Logistic Regression to predict product given review sentiment scores

In [153]:
lab = LabelEncoder()
dff3["Product_code"]= lab.fit_transform(dff3["Product_code"])
dff3

C:\Users\DELL\AppData\Local\Temp\ipykernel_12912\137515933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff3["Product_code"]= lab.fit_transform(dff3["Product_code"])


,rating,country,Product,neg,neu,pos,comp,Product_code
1,4.0,India,JBL Tune 710BT,0.000,0.822,0.178,0.3818,0
2,1.0,India,JBL Tune 710BT,0.629,0.371,0.000,-0.5256,0
3,2.0,India,JBL Tune 710BT,0.279,0.721,0.000,-0.7340,0
4,4.0,India,JBL Tune 710BT,0.000,1.000,0.000,0.0000,0
5,1.0,India,JBL Tune 710BT,0.000,0.580,0.420,0.4404,0
...,...,...,...,...,...,...,...,...
196,5.0,India,Sony Wh-Ch510,0.000,0.476,0.524,0.7650,1
197,1.0,India,Sony Wh-Ch510,0.048,0.827,0.126,0.8047,1
198,4.0,India,Sony Wh-Ch510,0.000,0.326,0.674,0.7351,1
199,5.0,India,Sony Wh-Ch510,0.000,0.791,0.209,0.7003,1


In [190]:
X=dff3[['neg','pos','neu','comp']]
Y=dff3['Product_code']

In [191]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=2)

In [212]:
arr = np.logspace(-3, 3, num=201)

In [213]:
model = LogisticRegression()
param={'penalty':['l1','l2','elasticnet'],'C':arr}

In [214]:
k_fold = KFold(n_splits=8, shuffle=True, random_state=1)
cv=cross_val_score(model,x_train,y_train,cv=k_fold)
print(np.mean(cv))

0.690625


In [215]:
classify=GridSearchCV(model, param_grid=param, scoring='accuracy',cv=k_fold)

In [216]:
classify.fit(x_train,y_train)

D:\Downloads\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
3216 fits failed out of a total of 4824.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1608 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Downloads\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Downloads\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "D:\Downloads\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError

GridSearchCV(cv=KFold(n_splits=8, random_state=1, shuffle=True),
             estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 1.07151931e-03, 1.14815362e-03, 1.23026877e-03,
       1.31825674e-03, 1.41253754e-03, 1.51356125e-03, 1.62181010e-03,
       1.73780083e-03, 1.86208714e-03, 1.99526231e-03, 2.13796209e-03,
       2.29086765e-03, 2.45470892e-03, 2.63026799e-03, 2.8183829...
       2.51188643e+02, 2.69153480e+02, 2.88403150e+02, 3.09029543e+02,
       3.31131121e+02, 3.54813389e+02, 3.80189396e+02, 4.07380278e+02,
       4.36515832e+02, 4.67735141e+02, 5.01187234e+02, 5.37031796e+02,
       5.75439937e+02, 6.16595002e+02, 6.60693448e+02, 7.07945784e+02,
       7.58577575e+02, 8.12830516e+02, 8.70963590e+02, 9.33254301e+02,
       1.00000000e+03]),
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [217]:
print(classify.best_params_)

{'C': 0.0630957344480193, 'penalty': 'l2'}


In [218]:
print(classify.best_score_)

0.73125


In [219]:
model.fit(x_train, y_train)

LogisticRegression()

In [220]:
y_pred=classify.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.775

In [221]:
y_pred

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=int64)

In [222]:
y_test

95     0
33     0
26     1
158    0
157    1
      ..
189    0
24     1
66     0
63     1
187    0
Name: Product_code, Length: 80, dtype: int64

In [223]:
confusion_matrix(y_test,y_pred)

array([[32, 10],
       [ 8, 30]], dtype=int64)